In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install -q torch==1.8 torchvision torchtext
!pip install -q pytorch-lightning==1.4.9 torchmetrics
!pip install transformers

In [ ]:
import pathlib
import json
import glob
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import AdamW
from transformers import BertConfig, BertForMaskedLM, BertModel, BertForSequenceClassification
# import pytorch_lightning as pl
# from pytorch_lightning import loggers as pl_loggers
# from pytorch_lightning.callbacks import ModelCheckpoint


In [ ]:

# TRAIN_LOG_DIRECTORY='drive/MyDrive/dl/wahaha/log'
TRAIN_LOG_DIRECTORY='drive/MyDrive/dl/wahaha/log2'
TRAIN_MLM_MODEL_DIR = 'drive/MyDrive/dl/wahaha/model'
TRAIN_PRETRAIN_MODEL_BIN = 'drive/MyDrive/dl/wahaha/model/pretrain_mlm/pytorch_model.bin'

TRAIN_FILE='drive/MyDrive/dl/wahaha/train/train_feature.txt'
TEST_FILE='drive/MyDrive/dl/wahaha/train/test_feature.txt'
MINI_TEST_FILE='drive/MyDrive/dl/wahaha/train/mini_test_feature_100.txt'
# TRAIN_DATA_DIRECTORY='drive/MyDrive/dl/kanachan/train'
# TEST_DATA_DIRECTORY='drive/MyDrive/dl/kanachan/test'

TRAIN_POSSIBLE_LABEL_COUNT = 219 # label count

TRAIN_TOKEN_PADDING = 0 #2 + 382 + 648 + 219 # 2(CLS,SEP)+382(sparse)+648(progression)+219(possible)
TRAIN_TOKEN_CLS     = 1
TRAIN_TOKEN_SEP     = 2
TRAIN_TOKEN_EOS = 3
TRAIN_TOKEN_MASK    = 4
TRAIN_TOKEN_START   = 6
TRAIN_TOKEN_VOCAB_COUNT = 974 # TRAIN_TOKEN_PADDING + 2 # 2(PADDING, MASK)
TRAIN_MAX_TOKEN_LENGTH = 2 + 139 + 1 # 26(sparse)+81(progression)+32(possible)
TRAIN_ID_LIST = list(range(TRAIN_TOKEN_START, TRAIN_TOKEN_VOCAB_COUNT))

TRAIN_HIDDEN_SIZE = 768

In [ ]:
# https://github.com/PyTorchLightning/pytorch-lightning/issues/2534#issuecomment-674582085
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename
    def save_checkpoint(self) :
        if self.use_modelcheckpoint_filename:
            filename = trainer.checkpoint_callback.filename
        else:
            #filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            filename = f"{self.prefix}.ckpt"
        ckpt_path = os.path.join(TRAIN_LOG_DIRECTORY, filename)
        trainer.save_checkpoint(ckpt_path)

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        # epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            self.save_checkpoint()

    def on_epoch_end(self, trainer, _ = None) :
        self.save_checkpoint()
class TrainDataset_old :
    """
     Read train dataset from text file 
    """
    def __init__(self, filename) :
        self.filename = filename
        with open(filename, 'r') as file :
            print("[TrainDataSet]--- start setup ---", filename)
            self.content = file.readlines()
            print("[TrainDataSet]--- loading is completed ---", filename)
    def __len__(self) :
        return len(self.content)
    def __getitem__(self, idx) :
        line = self.content[idx]
        inputs, label = line.split(',')
        inputs = inputs.split(' ')
        inputs = [int(n) for n in inputs]
        label = int(label)
        return inputs, label    
class TrainDataset :
    """
     Read train dataset from text file 
    """
    def __init__(self, filename) :
        self.filename = filename
        with open(filename, 'r') as file :
            print("[TrainDataSet]--- start setup ---", filename)
            self.content = file.readlines()
            print("[TrainDataSet]--- loading is completed ---", filename)
    def __len__(self) :
        return len(self.content)
    def __getitem__(self, idx) :
        line = self.content[idx]
        inputs, label = line.split(',')
        inputs = inputs.split(' ')
        inputs = [int(n) for n in inputs]
        label = int(label)
        return inputs, label

## mlm

In [ ]:
TRAIN_MLM_CONFIG = {
    'vocab_size': TRAIN_TOKEN_VOCAB_COUNT,  # MASK_TOKEN_ID, MASK, CLS, SEP
    'hidden_size': 768,
    'num_hidden_layers': 12,
    'num_attention_heads': 12,
    'intermediate_size': 3072,  # hidden_size * 4が目安
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'attention_probs_dropout_prob': 0.1,
    'max_position_embeddings': 512,  # 95(=81(マス目)+7(先手持駒)+7(後手持駒))でいいかも
    'type_vocab_size': 1,  # 対の文章を入れない。つまりtoken_type_embeddingsは完全に無駄になっている。
    'initializer_range': 0.02,
}

TRAIN_MLM_MODEL_CONFIG = { 
    'checkpoint_callback' : False, 
    #'gpus' : -1, # 環境にあわせて変更
    #'tpu_core' : 8,
    #'val_check_interval' : 5000, # validationおよびcheckpointの間隔step数
    #'max_epochs' : 5,
    'model_dir' : None, # インポートするモデル
}
TRAIN_MLM_TRAINER_CONFIG = {
    'checkpoint_callback' : False, 
    #'gpus' : -1, # 環境にあわせて変更
    'tpu_cores' : 8,
    'val_check_interval' : 5000, # validationおよびcheckpointの間隔step数
    'max_epochs' : 5
}
TRAIN_MLM_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 32,
    'shuffle' : True,
    'pin_memory' : False,
    'drop_last' : True
}
VAL_MLM_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 8,
    'pin_memory' : False,
    'drop_last' : False
}


In [ ]:
TRAIN_MLM_MODEL_CONFIG = BertConfig.from_dict(TRAIN_MLM_CONFIG)
class MLMModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.model = BertForMaskedLM(TRAIN_MLM_MODEL_CONFIG)

    def forward(self, batch):
        input_ids = batch['input_ids']
        labels = batch['labels']
        attention_mask = batch['attention_mask']
        return self.model(input_ids=input_ids, labels=labels, attention_mask = attention_mask)

    def training_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs[0]
        self.log('loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs[0].detach().cpu().numpy()
        return {'loss': loss}

    def validation_epoch_end(self, outputs):
        val_loss = np.mean([out['loss'] for out in outputs])
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)
        
    def test_epoch_end(self, outputs):
        self.validation_epoch_end(outputs)

class MLMDataset(Dataset):
    def __init__(self, filename):
        self.content = TrainDataset(filename)

    def __len__(self):
        return len(self.content)

    def __getitem__(self, idx):
        inputs, _ = self.content[idx]
        inputs = np.array(inputs)
        labels = inputs.copy()

        # 予想対象
        masked_indices = np.random.random(labels.shape) < 0.15
        labels[~masked_indices] = -100

        # 80%はマスクトークンに
        indices_replaced = (np.random.random(labels.shape) < 0.8) & masked_indices
        inputs[indices_replaced] = 0 # [MASK_ID]

        # 10%はランダムに置き換え
        indices_random = (np.random.random(labels.shape) < 0.5) & masked_indices & ~indices_replaced
        random_words = np.random.choice(TRAIN_ID_LIST, labels.shape)
        inputs[indices_random] = random_words[indices_random]
        
        # attention mask
        pad_length = TRAIN_MAX_TOKEN_LENGTH - len(inputs)
        attention_mask = np.array([1] * len(inputs) + [0] * pad_length)
        
        # padding
        inputs = np.pad(inputs,(0, pad_length), constant_values = 0) # [PAD]
        labels = np.pad(labels,(0, pad_length), constant_values = -100)

        ret_dict = {'input_ids': torch.tensor(inputs, dtype=torch.long),
                    'labels': torch.tensor(labels, dtype=torch.long),
                    'attention_mask' : torch.tensor(attention_mask, dtype=torch.float)
                   }
        return ret_dict

class MLMDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage=None) :
        self.train_dataset = MLMDataset(TRAIN_FILE)
        self.val_dataset = MLMDataset(TEST_FILE)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, **TRAIN_MLM_LOADER_CONFIG)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, **VAL_MLM_LOADER_CONFIG)


In [ ]:
hparams = TRAIN_MLM_MODEL_CONFIG
#trainer = pl.Trainer(gpus = hparams["gpus"], enable_checkpointing=hparams["enable_checkpointing"], max_epochs = hparams["max_epochs"], val_check_interval = hparams["val_check_interval"])
#trainer = pl.Trainer(tpu_cores=8, checkpoint_callback=hparams["enable_checkpointing"], max_epochs = hparams["max_epochs"], val_check_interval = hparams["val_check_interval"])
trainer = pl.Trainer(**TRAIN_MLM_TRAINER_CONFIG)

# trainer.logger = pl_loggers.TensorBoardLogger(save_dir=TRAIN_LOG_DIRECTORY, name="mlm", default_hp_metric=False)
# trainer.callbacks.append(CheckpointEveryNSteps(save_step_frequency = 5000, prefix="chk_mlm"))
### trainer.callbacks.append(ModelCheckpoint(filename='', monitor='val_loss', save_top_k=1, save_last=True))

model = MLMModule(TRAIN_MLM_MODEL_CONFIG)
#model = MLMModule.load_from_checkpoint(TRAIN_LOG_DIRECTORY + '/chk_mlm_220108a.ckpt')

data = MLMDataModule()
trainer.fit(model, datamodule=data)

result = trainer.test()

In [ ]:
from tempfile import mkdtemp
TRAIN_DATA_MAX_LEN_TOKEN = 112

class TrainDataset :
    """
     Read train dataset from text file 
    """
    def __init__(self, filename) :
        print("[TrainDatasetMmap]--- start setup ---", filename)

        with open(filename, 'r') as file :
            content = file.readlines()
            
            temp_filename = os.path.join(mkdtemp(), 'train_data.dat')
            print("tempfile:", temp_filename)
            mmap_shape = (len(content), TRAIN_DATA_MAX_LEN_TOKEN + 2) # label, len(inputs)分を加算
            print(mmap_shape)
            
            blob_content = np.empty(mmap_shape, dtype = "int32")
            print(blob_content)

            for idx, line in enumerate(content) :
                inputs, label = line.split(',')
                inputs = inputs.split(' ')
                inputs = [int(n) for n in inputs]
                label = int(label)

                # 最適化 : inputs, labelを一次元配列にエンコードする
                # [label, len(inputs), inputs[0], ...]
                
                pad_length = TRAIN_DATA_MAX_LEN_TOKEN - len(inputs)
                # 長さをそろえるためpaddingを付与
                inputs = np.pad(inputs,(0, pad_length), constant_values = 0)
                data = np.concatenate([ [label, len(inputs)], inputs])
                blob_content[idx] = data

            fp = np.memmap(temp_filename, dtype='uint32', mode='w+', shape = mmap_shape)
            fp[:] = blob_content[:]
            del fp
            
        self.mmap_content = np.memmap(temp_filename, dtype='uint32', mode='r', shape = mmap_shape)
        del content # free

        print("[TrainDatasetMmap]--- loading is completed ---", filename)

    def __len__(self) :
        return self.mmap_content.shape[0]  # len(self.mmap_content)
    def __getitem__(self, idx) :
        data = self.mmap_content[idx]
        label = data[0]
        length = data[1]
        inputs = data[2:length]
        return (inputs, label)

# dataset = MLMDataset('../mini_sample.txt')
dataset = TrainDataset_old('../mini_sample.txt')
print(f"load data len={len(dataset)}")
print(f"output[100] = {dataset[100]}")
dataset2 = TrainDataset('../mini_sample.txt')
print(f"load data len={len(dataset2)}")
print(f"output[100] = {dataset2[100]}")


In [ ]:

# Positional Embededを一部だけにするテスト
# MLM
# inputs = [1, 7, 8, 11, 14, 17, 21, 72, 169, 218, 256, 267, 269, 270, 279, 282, 285, 285, 286, 288, 289, 289, 290, 304, 2, 330,331,380,500,3]
inputs = [1, 7, 8, 11, 14, 17, 21, 72, 169, 218, 256, 267, 269, 270, 279, 282, 285, 285, 286, 288, 289, 289, 290, 304, 2, 3]
inputs = np.array(inputs)
labels = inputs.copy()
# print("input:",len(inputs), inputs)
# positional ids
positional_idx_start = np.where(inputs == TRAIN_TOKEN_SEP)[0][0] - 1
positional_arange = np.arange(labels.shape - positional_idx_start)
positional_ids = np.zeros(labels.shape)
positional_ids[positional_idx_start:] = positional_arange
# print("pos",len(positional_ids),positional_ids)

# 予想対象
masked_indices = np.random.random(labels.shape) < 0.15
labels[~masked_indices] = -100

# 80%はマスクトークンに
indices_replaced = (np.random.random(labels.shape) < 0.8) & masked_indices
inputs[indices_replaced] = TRAIN_TOKEN_MASK # [MASK_ID]

# 10%はランダムに置き換え
indices_random = (np.random.random(labels.shape) < 0.5) & masked_indices & ~indices_replaced
random_words = np.random.choice(range(TRAIN_TOKEN_START, TRAIN_TOKEN_VOCAB_COUNT), labels.shape)
# print(indices_random)
# print(random_words)
inputs[indices_random] = random_words[indices_random]

# attention mask
pad_length = TRAIN_MAX_TOKEN_LENGTH - len(inputs)
attention_mask = np.array([1] * len(inputs) + [0] * pad_length)

# padding
inputs = np.pad(inputs,(0, pad_length), constant_values = TRAIN_TOKEN_PADDING) # [PAD]
labels = np.pad(labels,(0, pad_length), constant_values = -100)
positional_ids = np.pad(positional_ids, (0, pad_length), constant_values = 0)

print("input:",len(inputs), inputs)
print("labels:",len(labels), labels)
print("attention:",len(attention_mask),attention_mask)
print("positional_ids:",len(positional_ids),positional_ids)
ret_dict = {'input_ids': torch.tensor(inputs, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long),
            'attention_mask' : torch.tensor(attention_mask, dtype=torch.float),
            'position_ids'  : torch.tensor(positional_ids, dtype=torch.long)
            }


In [ ]:
import string
import itertools

suffixes = list(map(''.join, itertools.product(string.ascii_lowercase, repeat = 2) ))
print(len(suffixes),suffixes)

In [ ]:
# TPU学習結果テスト
hparams = TRAIN_MLM_MODEL_CONFIG
trainer = pl.Trainer(**TRAIN_MLM_TRAINER_CONFIG)

#model = MLMModule(TRAIN_MLM_MODEL_CONFIG)
model = MLMModule.load_from_checkpoint(TRAIN_LOG_DIRECTORY + '/chk_mlm_220108a.ckpt')
ckpt_path = TRAIN_LOG_DIRECTORY + '/chk_mlm.ckpt'
model = MLMModule.load_from_checkpoint(ckpt_path)

# data = MLMDataModule()
# Test Dataset
dataset = MLMDataset(TEST_FILE)
dataloader = DataLoader(dataset, **VAL_MLM_LOADER_CONFIG)
# start test
result = trainer.test(model = model, dataloaders = dataloader, ckpt_path =ckpt_path)
print(result)


In [ ]:
# GPU学習結果テスト
hparams = TRAIN_MLM_MODEL_CONFIG
trainer = pl.Trainer(**TRAIN_MLM_TRAINER_CONFIG)

mlm_dirconfig = { 
    'model_dir' : TRAIN_MLM_MODEL_DIR + "/mlm_220108a.bin", # インポートするモデル
}
model = MLMModule(mlm_dirconfig)

# Test Dataset
dataset = MLMDataset(TEST_FILE)
dataloader = DataLoader(dataset, **VAL_MLM_LOADER_CONFIG)
# start test
result = trainer.test(model = model, dataloaders = dataloader)
print(result)

## Classification

In [ ]:
#!echo "start" > start_test.txt

TRAIN_CLASSIFICATION_CONFIG = {
    'vocab_size': TRAIN_TOKEN_VOCAB_COUNT,  # MASK_TOKEN_ID, MASK, CLS, SEP
    'hidden_size': TRAIN_HIDDEN_SIZE,
    'num_hidden_layers': 12,
    'num_attention_heads': 12,
    'intermediate_size': 3072,  # hidden_size * 4が目安
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'attention_probs_dropout_prob': 0.1,
    'max_position_embeddings': 512,  # 95(=81(マス目)+7(先手持駒)+7(後手持駒))でいいかも
    'type_vocab_size': 1,  # 対の文章を入れない。つまりtoken_type_embeddingsは完全に無駄になっている。
    'initializer_range': 0.02,
    'num_labels' : TRAIN_POSSIBLE_LABEL_COUNT, 
}
TRAIN_CLASSIFICATION_MODEL_CONFIG = {
    'checkpoint_callback' : False, 
    # 'gpus' : 1, # 環境にあわせて変更
    'tpu_cores' : 8,
    'val_check_interval' : 5000, # validationおよびcheckpointの間隔step数
    'max_epochs' : 3,
    #'model_dir' : TRAIN_MLM_MODEL_DIR, # インポートするモデル
    # 'enable_checkpointing' : False, 
}
TEST_CLASSIFICATION_MODEL_CONFIG = {
    # 'gpus' : 1, # 環境にあわせて変更
    'tpu_cores' : 1,
}
TRAIN_CLASSIFICATION_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 32,
    'shuffle' : True,
    'pin_memory' : True,
    'drop_last' : True
}
VAL_CLASSIFICATION_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 8,
    'pin_memory' : True,
    'drop_last' : False
}
TEST_CLASSIFICATION_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 8,
    'drop_last' : False
}
BERT_CLASSIFICATION_CONFIG = BertConfig.from_dict(TRAIN_CLASSIFICATION_CONFIG)

class BertClassification(nn.Module) :
    def __init__(self, model_dir):
        super().__init__()
        if model_dir is None :
            self.bert = BertForSequenceClassification(BERT_CLASSIFICATION_CONFIG)
        else :
            model_bin = model_dir
            self.bert = BertForSequenceClassification.from_pretrained(model_bin, config = BERT_CLASSIFICATION_CONFIG)

    def forward(self, input_ids, labels, attention_mask):
        outputs = self.bert(input_ids, labels = labels, attention_mask = attention_mask)
        loss = outputs[0]
        logit = outputs[1]
        return {'loss' : loss, 'logit' : logit}

class BertClassificationModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        model_dir = hparams["model_dir"]
        self.save_hyperparameters(hparams)
        self.model = BertClassification(model_dir)

    def forward(self, input_ids, labels, attention_mask):
        return self.model(input_ids=input_ids, labels=labels, attention_mask = attention_mask)

    def training_step(self, batch, batch_idx):
        input_ids = batch.pop('input_ids')
        labels = batch.pop('labels')
        attention_mask = batch.pop('attention_mask')
        outputs = self(input_ids, labels, attention_mask)
        loss = outputs['loss']
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch.pop('input_ids')
        labels = batch.pop('labels')
        attention_mask = batch.pop('attention_mask')
        outputs = self(input_ids, labels, attention_mask)
        loss = outputs['loss']
        return loss

    def validation_epoch_end(self, outputs):
        val_loss = np.mean([out.to('cpu').detach().numpy() for out in outputs])
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)
    
    def test_step(self, batch, batch_idx): #testの実行；最終的なlossとaccの確認
        input_ids = batch.pop('input_ids')
        labels = batch.pop('labels')
        attention_mask = batch.pop('attention_mask')
        outputs = self(input_ids, labels, attention_mask)
        loss = outputs['loss']
        logit = outputs['logit']        
        prediction = torch.argmax(logit, dim=1)
        
        #print("logit:", logit, ",label:", labels)

        return {'loss' : loss, 'pred' : prediction, 'label' : labels}

    def test_epoch_end(self, outputs):
        epoch_preds = torch.cat([x['pred'] for x in outputs])
        epoch_labels = torch.cat([x['label'] for x in outputs])
        epoch_labels = torch.flatten(epoch_labels)
        epoch_loss = np.mean([x['loss'] for x in outputs])
        self.log("test_loss", epoch_loss)

        # accuracy計算
        num_correct = (epoch_preds == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"test_accuracy", epoch_accuracy)

class BertClassificationDataset(Dataset):
    def __init__(self, filename):
        self.train_dataset = TrainDataset(filename)

    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        inputs, label = self.train_dataset[idx]
        inputs = [TRAIN_TOKEN_CLS] + inputs + [TRAIN_TOKEN_SEP]
        
        inputs = np.array(inputs)
        label = [label]
        
        # attention mask
        pad_length = TRAIN_MAX_TOKEN_LENGTH - len(inputs)
        attention_mask = np.array([1] * len(inputs) + [0] * pad_length)
        
        # padding
        inputs = np.pad(inputs,(0, pad_length), constant_values = TRAIN_TOKEN_PADDING)

        ret_dict = {'input_ids': torch.tensor(inputs, dtype=torch.long),
                    'labels': torch.tensor(label, dtype=torch.long),
                    'attention_mask' : torch.tensor(attention_mask, dtype = torch.float)
                    }
        return ret_dict

class BertClassificationDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage=None) :
        self.train_dataset = BertClassificationDataset(TRAIN_FILE)
        self.val_dataset = BertClassificationDataset(TEST_FILE)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, **TRAIN_CLASSIFICATION_LOADER_CONFIG)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, **VAL_CLASSIFICATION_LOADER_CONFIG)
    
    def test_dataloader(self) : 
        # wip
        return DataLoader(self.val_dataset, **TEST_CLASSIFICATION_LOADER_CONFIG)


In [ ]:
# Train

trainer = pl.Trainer(**TRAIN_CLASSIFICATION_MODEL_CONFIG)
trainer.logger = pl_loggers.TensorBoardLogger(save_dir=TRAIN_LOG_DIRECTORY, name="wahaha_classification", default_hp_metric=False)
trainer.callbacks.append(CheckpointEveryNSteps(save_step_frequency = 5000, prefix="chk_class"))

# Model
mlm_dirconfig = { 
    'model_dir' : TRAIN_MLM_MODEL_DIR + "/mlm_220108a.bin", # インポートするモデル
}
model = BertClassificationModule(mlm_dirconfig) 
# model = BertClassificationModule.load_from_checkpoint(TRAIN_LOG_DIRECTORY + '/N-Step-Checkpoint.ckpt')

data = BertClassificationDataModule()
trainer.fit(model, datamodule=data)


In [ ]:
from torchinfo import summary

model = BertForMaskedLM(TRAIN_MLM_MODEL_CONFIG)
inshape = (1, 10)
label = torch.LongTensor(np.array([[0]*10]))
summary(model, input_size=inshape, labels = label , dtypes=[torch.long], depth = 4)

In [ ]:
model = BertForSequenceClassification(BERT_CLASSIFICATION_CONFIG)
for name, param in model.named_parameters(recurse=True) :
    print(name, param.size(), param.requires_grad )

In [ ]:
possible_action_ids = np.array([1,4,7,8,20])
possible_action_ids = np.unique(possible_action_ids)
# possible_actionのindexでlogitをマスクする
logit = np.array([0.3] * 100)
mask = np.zeros(logit.shape, dtype=bool)
print(possible_action_ids.dtype)
mask[possible_action_ids] = True
masked_logit = np.ma.masked_array(logit, mask = ~mask)


In [ ]:
# pandas dataset

class TrainDataset :
    """
     Read train dataset from text file 
    """
    def __init__(self, filename) :
        self.filename = filename
        with open(filename, 'r') as file :
            print("[TrainDataSet]--- start setup ---", filename)
            self.content = file.readlines()
            print("[TrainDataSet]--- loading is completed ---", filename)
    def __len__(self) :
        return len(self.content)
    def __getitem__(self, idx) :
        line = self.content[idx]
        inputs, label = line.split(',')
        inputs = inputs.split(' ')
        inputs = [int(n) for n in inputs]
        label = int(label)
        return inputs, label

# import pandas as pd

class TrainDatasetEx :
    """
     Read train dataset from text file 
    """
    def __init__(self, filename) :
        self.filename = filename
        with open(filename, 'r') as file :
            print("[TrainDataSet]--- start setup ---", filename)
            self.content = file.readlines()
            print("[TrainDataSet]--- loading is completed ---", filename)
    def __len__(self) :
        return len(self.content)
    def __getitem__(self, idx) :
        line = self.content[idx]
        inputs, label = line.split(',')
        inputs = inputs.split(' ')
        inputs = [int(n) for n in inputs]
        label = int(label)
        return inputs, label
filename = '../mini_sample.txt'


dataset = TrainDataset(filename, chunksize = 100)
idx = 10
dataset_ex = TrainDatasetEx(filename)
assert dataset[100] == dataset_ex[100], "test fail"
print(f"data[{idx}] = {dataset[idx]}\n len = {len(dataset)}")


In [ ]:
class Trainer :
    def __init__(self) :
        self.current_epoch = 0
class EpochDataset :
    """
     Epochごとに異なるファイルを読み出すDataset
    """
    # def __init__(self, trainer : pl.Trainer) :
    def __init__(self, trainer, prefix = 'random/random_data_a_', suffix_list = list('bcdefghijklmnopqrstuvw')) :
        self.trainer = trainer # 現在epochを参照するために保持
        self.content_index = 0 # 現在読み出しているcontent
        self.load_file(self.content_index) # 最初は0番目のファイルを読んでおく

    def __len__(self) :
        return len(self.content)

    def __getitem__(self, idx) :
        idx = self.trainer.current_epoch % len(self.suffix_list)
        if self.content_index != idx :
            # epochが切り替わったとき、ファイルを読み出す。
            self.content = self.load_file(idx)
        line = self.content[idx]
        inputs, label = line.split(',')
        inputs = inputs.split(' ')
        inputs = [int(n) for n in inputs]
        label = int(label)
        return inputs, label

    def load_file(self, index) :
        filename = self.prefix + self.suffix_list[index]
        with open(filename, 'r') as file :
            print("[TrainDatasetEx] load file :",filename)
            return file.readlines()

suffix_list = list('bcdefghijklmnopqrstuvw')
print(suffix_list)




In [ ]:


def convert_pai_sangen(pai, shift_count) :
    # 三元牌をシフト
    SANGENPAI = ["P", "F", "C"]
    if pai in SANGENPAI :
        idx = SANGENPAI.index(pai)
        pai = SANGENPAI[(idx + shift_count) % len(SANGENPAI)]
    return pai

def convert_pai_ms(pai) :
    # ピンズとソーズを入れ替え
    if pai.find("p") > 0 :
        pai = pai.replace("p", "s")
    elif pai.find("s") > 0 :
        pai = pai.replace("s", "p")
    return pai

def convert_pai_manzu(pai) :
    # サンマのみ: 1mと9mを入れ替え
    MANZU_TABLE = {"1m" : "9m", "9m" : "1m"}
    if pai in MANZU_TABLE :
        pai = MANZU_TABLE[pai]
    return pai

def create_converter(swap_ms = False, swap_manzu = False, shift_count_sangen = 0) :
    res = lambda p : convert_pai_sangen(p, shift_count_sangen)
    if swap_ms :
        res = lambda p : convert_pai_ms(res(p))
        print(res)
    # if swap_manzu :
    #     res = lambda p : convert_pai_manzu(res(p))
    return res

TEST_PATTERN = [
    {"swap_ms" : True, "swap_manzu" : False, "shift_count_sangen" : 0}
]
for pattern in TEST_PATTERN :
    # converter = create_converter(**pattern)
    converter = create_converter(True)
    print_convert = lambda p : print(p , "=>", converter(p) )
    print_convert("1m")
    print_convert("9m")
    print_convert("1p")
    print_convert("5pr")
    print_convert("5sr")
    print_convert("0m")
    print_convert("P")
    print_convert("F")
    print_convert("C")

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score,confusion_matrix

filename = r'D:\home\program\maj3\wahaha_bert\_tmp\test_result.npz'

npz_arr = np.load(filename, allow_pickle=True)
input_ids = npz_arr["arr_0"]
labels = npz_arr["arr_1"]
preds = npz_arr["arr_2"]
# print("input:", input_ids.shape, input_ids)
print("label:", labels.shape, labels)
# print("pred :", preds.shape, preds)

hist = np.bincount(labels)
hist_pred = np.bincount(preds)
pre_score = precision_score(labels, preds, labels = range(0,112), average = None)
for i, (count, count_pred, score) in enumerate(zip(hist, hist_pred, pre_score)) :
    print(f"{i}\t{count}\t{count_pred}\t{score}")

conf = confusion_matrix(labels, preds, labels = range(0,112))
pd.set_option('display.max_rows', 128)
pd.set_option('display.max_columns', 128)
pd.DataFrame(conf)


label: (32768,) [ 28 111  29 ...  35 109 111]
0	0	0	0.0
1	650	707	0.8048090523338048
2	0	0	0.0
3	0	0	0.0
4	0	0	0.0
5	0	0	0.0
6	0	0	0.0
7	0	0	0.0
8	0	0	0.0
9	640	687	0.7962154294032023
10	39	27	0.9629629629629629
11	425	434	0.7626728110599078
12	292	276	0.7789855072463768
13	226	195	0.8
14	200	173	0.7976878612716763
15	136	134	0.753731343283582
16	188	168	0.75
17	216	194	0.7938144329896907
18	268	283	0.7031802120141343
19	398	390	0.735897435897436
20	18	21	0.6666666666666666
21	478	447	0.8053691275167785
22	300	268	0.8022388059701493
23	218	206	0.7912621359223301
24	199	189	0.7513227513227513
25	144	150	0.7333333333333333
26	187	184	0.8097826086956522
27	221	221	0.7601809954751131
28	318	304	0.7828947368421053
29	444	429	0.7762237762237763
30	555	576	0.796875
31	557	546	0.8278388278388278
32	676	691	0.8219971056439942
33	16	10	0.8
34	528	563	0.7548845470692718
35	596	575	0.8121739130434783
36	546	570	0.8
37	211	244	0.6967213114754098
38	2708	1131	0.618921308576481
39	18	3	0.666666666666

d:\home\program\maj3\_venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,569,0,0,0,0,0,0,0,12,0,5,2,1,1,2,0,2,0,1,0,2,2,2,1,0,1,1,1,3,11,5,14,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,18,0,0,0,0,0,0,0,547,0,2,0,0,1,1,4,2,2,4,0,0,3,1,2,2,3,2,1,6,8,4,18,0,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
